# Data Analysis with Spark (sparkSQL) & hadoop using Python

In [1]:
import findspark
findspark.init()

# importing essential libraries
from pyspark import *
import pyspark.sql as sql
from pyspark.sql import *

# creating spark session (entry point of PySpark)
ss = sql.SparkSession.builder.appName('testing').getOrCreate()

## Creating custom Dataframe in PySpark

In [2]:
Employee = Row("firstName", "lastName", "email", "salary")

In [3]:
employee1 = Employee('Basher', 'armbrust', 'bash@edureka.co', 100000)
employee2 = Employee('Daniel', 'meng', 'daniel@stanford.edu', 120000 )
employee3 = Employee('Muriel', None, 'muriel@waterloo.edu', 140000 )
employee4 = Employee('Rachel', 'wendell', 'rach_3@edureka.co', 160000 )
employee5 = Employee('Zach', 'galifianakis', 'zach_g@edureka.co', 160000 )

department1 = Row(id='123456', name='HR')
department2 = Row(id='789012', name='OPS')
department3 = Row(id='345678', name='FN')
department4 = Row(id='901234', name='DEV')

In [4]:
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2, employee5])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4, employee3])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

In [5]:
departmentsWithEmployees_Seq = [departmentWithEmployees1, departmentWithEmployees2]

In [6]:
dframe = ss.createDataFrame(departmentsWithEmployees_Seq)

In [7]:
dframe.show()

+-------------+--------------------+
|   department|           employees|
+-------------+--------------------+
| [123456, HR]|[[Basher, armbrus...|
|[789012, OPS]|[[Muriel,, muriel...|
+-------------+--------------------+



## Importing External Data & making Dataframe (from Local)

In [8]:
dataset = ss.read.csv('/home/garvit/BigData Training Stuff/PySpark/Data_1/stock_data.csv',header=True)

In [9]:
dataset.show(2)

+----------+------+----+----+-----+-----+----+---------+---------+-----------+----------+-----------+------------+
|    SYMBOL|SERIES|OPEN|HIGH|  LOW|CLOSE|LAST|PREVCLOSE|TOTTRDQTY|  TOTTRDVAL| TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+----+----+-----+-----+----+---------+---------+-----------+----------+-----------+------------+
| 20MICRONS|    EQ|37.8|37.8|36.15|36.85|37.4|    37.05|    27130|   994657.9|2017-06-28|        202|INE144J01027|
|3IINFOTECH|    EQ| 4.1|4.85|  4.0| 4.55|4.65|     4.05| 20157058|92148517.65|2017-06-28|       7353|INE748C01020|
+----------+------+----+----+-----+-----+----+---------+---------+-----------+----------+-----------+------------+
only showing top 2 rows



In [10]:
dataset.printSchema()

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: string (nullable = true)
 |-- HIGH: string (nullable = true)
 |-- LOW: string (nullable = true)
 |-- CLOSE: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- PREVCLOSE: string (nullable = true)
 |-- TOTTRDQTY: string (nullable = true)
 |-- TOTTRDVAL: string (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: string (nullable = true)
 |-- ISIN: string (nullable = true)



In [11]:
dataset.count()

846404

In [12]:
dataset.columns

['SYMBOL',
 'SERIES',
 'OPEN',
 'HIGH',
 'LOW',
 'CLOSE',
 'LAST',
 'PREVCLOSE',
 'TOTTRDQTY',
 'TOTTRDVAL',
 'TIMESTAMP',
 'TOTALTRADES',
 'ISIN']

In [13]:
len(dataset.columns)

13

### describing a column using describe() function

In [14]:
dataset.describe('totaltrades').show()

+-------+------------------+
|summary|       totaltrades|
+-------+------------------+
|  count|            846404|
|   mean| 5013.823219171932|
| stddev|14901.118242887595|
|    min|                 1|
|    max|             99995|
+-------+------------------+



#### selecting multiple columns

In [15]:
dataset.select('symbol','totaltrades').show(5)

+----------+-----------+
|    symbol|totaltrades|
+----------+-----------+
| 20MICRONS|        202|
|3IINFOTECH|       7353|
|   3MINDIA|        748|
|   63MOONS|        437|
|   8KMILES|       1866|
+----------+-----------+
only showing top 5 rows



#### filtering data using filter() function

In [16]:
dataset.filter(dataset.SERIES=='EQ').count()

739199

#### grouping data using groupby() function

In [17]:
dataset.select('series','high').groupBy('high').avg().count()

58291

## Dataframe with SQL

In [18]:
dataset.show(2)

+----------+------+----+----+-----+-----+----+---------+---------+-----------+----------+-----------+------------+
|    SYMBOL|SERIES|OPEN|HIGH|  LOW|CLOSE|LAST|PREVCLOSE|TOTTRDQTY|  TOTTRDVAL| TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+----+----+-----+-----+----+---------+---------+-----------+----------+-----------+------------+
| 20MICRONS|    EQ|37.8|37.8|36.15|36.85|37.4|    37.05|    27130|   994657.9|2017-06-28|        202|INE144J01027|
|3IINFOTECH|    EQ| 4.1|4.85|  4.0| 4.55|4.65|     4.05| 20157058|92148517.65|2017-06-28|       7353|INE748C01020|
+----------+------+----+----+-----+-----+----+---------+---------+-----------+----------+-----------+------------+
only showing top 2 rows



#### registering "Dataframe" as  temperory "Table"

In [19]:
sql.SQLContext(sparkSession=ss, sparkContext=ss.sparkContext).registerDataFrameAsTable(dataset,'stock_data')
# enabling backword compability by passing parameters in SQLContext()

#### Writing SQL query

In [20]:
df2 = sql.SQLContext(sparkContext=ss.sparkContext, sparkSession=ss) \
.sql('select * from stock_data where TOTALTRADES=99995')

In [21]:
df2.show()

+---------+------+-----+-----+-----+------+------+---------+---------+-------------+----------+-----------+------------+
|   SYMBOL|SERIES| OPEN| HIGH|  LOW| CLOSE|  LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL| TIMESTAMP|TOTALTRADES|        ISIN|
+---------+------+-----+-----+-----+------+------+---------+---------+-------------+----------+-----------+------------+
|TATASTEEL|    EQ|327.5|336.2|326.0|332.85|332.55|    324.5| 10284415|3411250923.95|2016-04-13|      99995|INE081A01012|
+---------+------+-----+-----+-----+------+------+---------+---------+-------------+----------+-----------+------------+



In [22]:
sql.SQLContext(sparkSession=ss, sparkContext = ss.sparkContext) \
.sql('select SERIES,count(TOTALTRADES) as tt from stock_data group by SERIES order by tt desc').show()

+------+------+
|SERIES|    tt|
+------+------+
|    EQ|739199|
|    BE| 37428|
|    SM| 11229|
|    N6|  5080|
|    N2|  4828|
|    N4|  3722|
|    N5|  3661|
|    N1|  3576|
|    N3|  3098|
|    N8|  2884|
|    BZ|  2825|
|  null|  2457|
|    GB|  2343|
|    N9|  2021|
|    N7|  1658|
|    NE|  1602|
|    NB|  1467|
|    NC|  1195|
|    ND|  1046|
|    P2|   923|
+------+------+
only showing top 20 rows

